## **Memuat Model dan Processor Wav2Vec2**



In [ ]:
from transformers import pipeline

asr = pipeline("automatic-speech-recognition")

No model was supplied, defaulted to facebook/wav2vec2-base-960h and revision 55bb623 (https://huggingface.co/facebook/wav2vec2-base-960h).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/378M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You sho

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

**Mengunduh Dataset**

In [ ]:
!pip install datasets
from datasets import load_dataset
from datasets import Audio

dataset = load_dataset("jbpark0614/speechocean762", split="train")
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/2500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

**Sampling Audio**

In [ ]:
from IPython.display import Audio

sample = dataset[2]

print(sample["text"])
Audio(sample["audio"]["array"], rate=sample["audio"]["sampling_rate"])

THREE TWO TWO SEVEN


In [ ]:
txt = asr(sample["audio"].copy())

In [ ]:
reference = sample["text"]
print(reference)

THREE TWO TWO SEVEN


In [ ]:
prediction = txt["text"]
print(prediction)

THREE TWO TWO SEVEN


**Evaluation Metric**

In [ ]:
pip install --upgrade evaluate jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 15.7 MB/s eta 0:00:00


In [ ]:
from evaluate import load
from math import ceil
cer_metric = load("cer")

cer = cer_metric.compute(references=[reference], predictions=[prediction])
prc = ceil(cer*100)
print(prc)

0


In [ ]:
acc = 100 - prc
print(str(acc)+"%")

100%


# *Menyimpan Model hingga dapat memprediksi*

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import torch

# Load the model and processor
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Save the model state dictionary
torch.save(model.state_dict(), 'wav2vec2-base-960h.pt')

# Save the processor
processor.save_pretrained("wav2vec2_processor")

In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Config
import torch

# Create a configuration object for Wav2Vec2
config = Wav2Vec2Config()

# Instantiate the model using the configuration
model = Wav2Vec2ForCTC(config)

# Load your custom model weights
model.load_state_dict(torch.load('/content/wav2vec2-base-960h.pt', map_location=torch.device('cpu')))

# Set the model to evaluation mode
model.eval()

### Pra-proses input audio

In [ ]:
from pydub import AudioSegment

def resample_audio(audio_filepath, target_sampling_rate=16000):
    audio = AudioSegment.from_mp3(audio_filepath)
    audio = audio.set_frame_rate(target_sampling_rate)
    audio.export(audio_filepath, format="mp3")

audio_filepath = '/content/test.mp3'
resample_audio(audio_filepath)

### Fungsi Prediksi

In [ ]:
import torchaudio
from transformers import Wav2Vec2Processor

# Load the processor
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

def speech_file_to_array_fn(audio_filepath):
    speech_array, sampling_rate = torchaudio.load(audio_filepath)
    return speech_array.squeeze(0).numpy(), sampling_rate

def predict(audio_filepath):
    speech, sampling_rate = speech_file_to_array_fn(audio_filepath)
    inputs = processor(speech, sampling_rate=sampling_rate, return_tensors="pt", padding=True)

    input_values = inputs.input_values

    # Check if attention_mask is provided
    if "attention_mask" in inputs:
        attention_mask = inputs.attention_mask
        logits = model(input_values, attention_mask=attention_mask).logits
    else:
        logits = model(input_values).logits

    with torch.no_grad():
        predicted_ids = torch.argmax(logits, dim=-1)

    transcription = processor.batch_decode(predicted_ids)
    return transcription

### Contoh Prediksi

In [ ]:
# Example usage to predict
audio_filepath = '/content/test.mp3'
transcription = predict(audio_filepath)
print(transcription)

In [ ]:
input_text_reference = input('')

In [ ]:
reference = [input_text_reference.upper()]
print(reference)

### Evaluation Metric

In [ ]:
from evaluate import load
from math import ceil

# Load the CER metric
cer_metric = load("cer")

# Example usage
reference = str(reference)  # Replace with the actual ground truth text
transcription = str(transcription) # Replace with your model's predicted text

# Compute CER
cer = cer_metric.compute(references=[reference], predictions=[transcription])
prc = ceil(cer * 100)
print("CER:", prc, "%")

# Calculate Accuracy
acc = 100 - prc
print("Accuracy:", acc, "%")